In [9]:
import pandas as pd
import numpy as np
import random
import math 
import time
def prepare_data(name):
    x = np.loadtxt(name, usecols=[0,1,2,3,4,5,6])
    y = np.loadtxt(name, usecols=[7])
    return x, y, len(x)
# Import data
data,ylabels,n = prepare_data("seeds_dataset.txt")

Kmeans

In [10]:
## Compare
def Assignment_1(data,cList,labelDict,n,k):
    for i in range(n):
        disList = []
        for c in range(k):
            d = np.linalg.norm(data[i]-cList[c])
            disList.append(d)
        labelDict[i] = disList.index(min(disList))
    return labelDict

## Refitting
def refitting_1(data,cList,labelDict,n,k):
    error = 0
    for c in range(k):
        temp = np.array([0]*7)
        count = 0
        for i in range(n):
            if labelDict[i] == c:
                temp = data[i]+temp
                count += 1
        if count != 0:
            temp = temp/count
            error += np.linalg.norm(cList[c] - temp)
            cList[c] = temp
    if error == 0:
        return True
    else:
        return False

def Kmeans_Main(data, n):
    # Cluster Number
    k = 3
    # Label List
    labelDict = []
    for i in range(n):
        labelDict.append(0)
    cList = []
    for i in range(3):
        c = data[random.randint(0,n-1)]
        cList.append(c)
    Assignment_1(data,cList,labelDict,n,k)
    itr = 0
    while not refitting_1(data,cList,labelDict,n,k):
        itr += 1
        Assignment_1(data,cList,labelDict,n,k)
    cList = pd.DataFrame(cList)
    return labelDict, itr

GMM

In [11]:
def normalProbability(x,miuVectorK, covMatrixK):
    k = len(miuVectorK)
    cov_inv = np.linalg.inv(covMatrixK)
    delta = x - miuVectorK
    part =  np.dot(delta,cov_inv)
    part = -0.5*np.dot(part,delta)

    former = 1/math.sqrt( (2*math.pi)**k *np.linalg.norm(covMatrixK,2))
    return former*math.exp(part)

def covMCalculation(x,miuK):
    covMatrix = np.zeros((len(x),len(x)))
    delta = x - miuK
    for i in range(len(x)):
        for j in range(len(x)):
            covMatrix[i][j] = delta[i]*delta[j]
    return covMatrix

def Estep(data, gammaMatrix, piKVector, miuMatrix, covMatrixList, k,n):
    for i in range(n):
        for c in range(k):
            piK = piKVector[c]
            numerator =  piK*normalProbability(data[i],miuMatrix[c],covMatrixList[c])
            denominator = 0
            for j in range(k):
                denominator += piKVector[j]*normalProbability(data[i],miuMatrix[j],covMatrixList[j])
            gammaMatrix[i][c] = numerator / denominator

def Mstep(data,gammaMarix,k,n):
    piKVector = []
    miuMatrix = []
    covMatrixList = []
    for c in range(k):
        nK = 0
        miuK = np.zeros(7)
        covMatrixK = np.zeros((7,7))
        for i in range(n):
            nK += gammaMarix[i][c]
            miuK += np.multiply(data[i],gammaMarix[i][c])
        miuK = np.divide(miuK,nK)
        for i in range(n):
            covMatrixK += covMCalculation(data[i], miuK)*gammaMarix[i][c]
        covMatrixk = np.divide(covMatrixK, nK)
        piK = nK/n
        piKVector.append(piK)
        miuMatrix.append(miuK)
        covMatrixList.append(covMatrixk)
            
    return piKVector, miuMatrix, covMatrixList

def checkConvergence(data, piKVector, miuMatrix, covMatrixList, k,n):
    likelihood = 0
    for i in range(n):
        summation = 0
        for c in range(k):
            summation += piKVector[c]*normalProbability(data[i],miuMatrix[c],covMatrixList[c])
        likelihood += math.log(summation)
    return likelihood   

def processResult(gammaMatrix):
    labelDict = []
    for i in range(len(gammaMatrix)):
        labelDict.append(gammaMatrix[i].index(max(gammaMatrix[i])))
    return labelDict

def GMM_Main(data,n):
    itr = 0
    while True:
        try:
            k = 3
            # Initialize pik
            piKVector = [1/3, 1/3, 1/3]
            # Discover properties of miu and cov
            miuK = np.zeros(7)
            for i in range(n):
                miuK += data[i]
            miuK = np.divide(miuK,n)
            # Initialize miuMatrix
            miuMatrix = []
            for i in range(3):
                c1 = data[random.randint(0,n-1)]
                c2 = data[random.randint(0,n-1)]
                c3 = data[random.randint(0,n-1)]
                c = c1+c2+c3
                miuMatrix.append(np.divide(c,3))

            # Initialize covMatrixList
            covMatrixList = []
            for i in range(k):
                covM = []
                for ci in range(7):
                    cm = [0,0,0,0,0,0,0]
                    cm[ci] = 1
                    covM.append(cm)
                covMatrixList.append(covM)

            # Initialize Gamma
            gammaMatrix = []
            for i in range(n):
                gL = []
                for i in range(k):
                    gL.append(0)
                gammaMatrix.append(gL)
            Estep(data, gammaMatrix, piKVector, miuMatrix, covMatrixList, k,n)
            newlikelihood = checkConvergence(data, piKVector, miuMatrix, covMatrixList, k,n)
            likelihood = 0
            while abs(newlikelihood - likelihood) >= 0.1:
                itr += 1
                likelihood = newlikelihood
                Estep(data, gammaMatrix, piKVector, miuMatrix, covMatrixList, k,n)
                piKVector, miuMatrix, covMatrixList = Mstep(data,gammaMatrix,k,n)
                newlikelihood = checkConvergence(data, piKVector, miuMatrix, covMatrixList, k,n)
            return processResult(gammaMatrix), itr
        except:
            pass

Evaluation

In [12]:
# Silhoutte COeef
def single(data,index,labels):
    label_x=labels[index]
    
    b_list=[]
    for k in range(3):
        x_cluster=[]
        for i in range(data.shape[0]):
            if labels[i]==k:
                x_cluster.append(data[i])
        if label_x==k:
            distance=0
            for j in range(len(x_cluster)):
                distance+=np.linalg.norm(data[index]-x_cluster[j])
            a=distance/len(x_cluster)
        else:
            distance1=0
            for j in range(len(x_cluster)):
                distance1+=np.linalg.norm(data[index]-x_cluster[j])
            d=distance1/len(x_cluster)
            b_list.append(d)
    b=min(b_list)
    s=(b-a)/max(a,b)
    return s    
    
def Sil(data,labels):
    s = 0
    for index in range(data.shape[0]):
        s += single(data,index,labels)
    return s/data.shape[0]  

## Rand Index
def Rand(original,label): 
    a =0
    b =0
    n=len(original)
    for i in range(n-1):
        for j in range(i+1,n):
            if original[i] == original[j] and label[i] == label[j]:
                a+=1
            elif original[i] != original[j] and label[i] != label[j]:
                b+=1
            else:
                pass
    coef = (a+b) / (0.5*n*(n-1))
    return coef

In [13]:
# K-means Output
KmeansResult = Kmeans_Main(data,n)[0]
sc = Sil(data,KmeansResult)
ri = Rand(ylabels,KmeansResult)
print("Sihoutte Coefficient: ", sc)
print("Rand Index:", ri)

Sihoutte Coefficient:  0.47570673587797513
Rand Index: 0.8713602187286398


In [14]:
# GMM Output
GMMResult = GMM_Main(data,n)[0]
sc = Sil(data,GMMResult)
ri = Rand(ylabels,GMMResult)
print("Sihoutte Coefficient: ", sc)
print("Rand Index:", ri)

Sihoutte Coefficient:  0.3380434755952955
Rand Index: 0.7923900660742766
